In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import os
import re
import scipy as sci
import time

In [2]:
# global vars
samp_rt = 5120 #sampling rate Hz
time_import = 3000 # time to analyse, seconds
data_import = samp_rt * time_import # lines of data to import

In [3]:
def list_o_list(big_list, lil_size):
    l_o_l = []
    for i in range(0, len(big_list), lil_size):
        lil = big_list[i : min(i + lil_size, len(big_list))]
        l_o_l.append(lil)
    return(l_o_l)

In [4]:
def freq_analysis(sample , ttime, responsefrq =1, sampfrq = 5120):
#     mmtic = time.perf_counter()
    frqs = ""
    amps = []
    samp_int = 1/sampfrq
    ft = np.fft.fft(sample)/len(sample)
#     mmhic = time.perf_counter()
    ft = ft[range(int(len(sample)/2))]
    tpCount = len(sample)
    values = np.arange(int(tpCount/2))
    timePeriod = tpCount/sampfrq
    frequencies = values/timePeriod
    ft = abs(ft)
#     mmcup = time.perf_counter()
    #print(len(ft))
    #print(len(frequencies))
    feq = pd.DataFrame(data = {"freq":frequencies})
#     l1 = time.perf_counter()
    feq["amp"] = ft
#     l2 = time.perf_counter()
    frqs = np.arange(0, 1 + int(feq.freq.max()), responsefrq)
#     l3 = time.perf_counter()
    #print(len(frqs))
#     mmbless = time.perf_counter()
    wow = feq.amp.rolling(responsefrq).mean()
    yep = np.arange(responsefrq - 1, len(wow) + responsefrq  -1, responsefrq)
    amps = wow[yep]
#     l4 = time.perf_counter()
#     mmtoc = time.perf_counter()
#     print("init:", mmhic-mmtic)
#     print("fft:", mmcup - mmhic, "total:", mmcup - mmtic)
#     print("df1:", mmbless - mmcup, "total:", mmbless - mmtic)
#     print("done:", mmtoc - mmbless, "total:", mmtoc - mmtic)
#     print("l1, l2, l3, l4:", l1-mmcup, l2-mmcup, l3-mmcup, l4-mmcup)
#     print("for:", l4 - l3)
    return(frqs, amps)

In [70]:
def process(file):
    df = pd.read_csv(file.path, header=15, names=["time", "acceleration"], nrows = data_import)
    
    l1 = list(df.acceleration)
    
    
    l2 = list_o_list(l1, 5120)
    del l1
    
    bin_size = 10
    tot_frqs = int((samp_rt / 2) / bin_size)
    tic_a = time.perf_counter()
    ft_l1 = np.empty((time_import, tot_frqs))
    frq_l1 = np.empty(tot_frqs, np.int8)
    for c,v in enumerate(l2):
        frq_l1, amp = freq_analysis(v, 1, bin_size, samp_rt) # 1 is the total time of the sample (1 second)
        ft_l1[c] = amp
    toc_a = time.perf_counter()
    print("time diff:", toc_a - tic_a)
    direct = pd.DataFrame(data = ft_l1, columns = frq_l1)
    l2_df = pd.DataFrame(data =l2)
    del l2
    direct["mean_acc"] = l2_df.abs().mean(axis=1)
    direct["median"] = l2_df.abs().median(axis=1)
    direct["95_acc"] = l2_df.abs().quantile(.95)
    direct["max_acc"] = l2_df.abs().max(axis=1)
    del l2_df
    file_name = file.name[:-3] + "dat"
    direct.to_csv(file_name)
    toe = time.perf_counter()
    print("Finished in:", toe - tic_a, "seconds")
    return()
    
    
    
    
    
    

In [71]:
os.listdir()
home = os.getcwd()
dirs = []
dirs.append(home)
with os.scandir(home) as contents:
    for i in contents:
        if i.is_dir():
            dirs.append(i.path)
print(dirs)
                 


['C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\.git', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\.ipynb_checkpoints', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\01-08-19-V118', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\03-05-19-V118', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\03-26-19-V118', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\08-14-18-V118', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\GPS Files', 'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\Test Train Sample Noise Data']


In [ ]:
for i in dirs:
    
    with os.scandir(i) as stuff:
        for thing in stuff:
            if re.search(".csv", thing.name) and re.search("_ch\d", thing.name):
                print(thing.name)
                process(thing)
    


01-08-19-V118_ch1.csv
time diff: 19.801157200000034
Finished in: 31.698206500000197 seconds
01-08-19-V118_ch2.csv
time diff: 20.166382200000044
Finished in: 31.723734400000012 seconds
REC0001_ch1.csv
time diff: 20.14995500000032
Finished in: 31.777458200000183 seconds
REC0001_ch2.csv
time diff: 19.99009309999974
Finished in: 31.35782479999989 seconds
REC0002_ch1.csv
time diff: 19.79238400000031
Finished in: 31.294547600000442 seconds
REC0002_ch2.csv
time diff: 20.032706799999687
Finished in: 31.60263489999943 seconds
08-14-18-V118_ch1.csv
time diff: 19.991257900000164
Finished in: 31.678287000000637 seconds
08-14-18-V118_ch2.csv
